<a href="https://colab.research.google.com/github/AbdurNawaz/BlackJack-v0/blob/master/BlackJack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gym
import sys
import numpy as np
from collections import defaultdict
from plot_utils import plot_blackjack_values, plot_policy

In [0]:
env = gym.make('Blackjack-v0')

In [26]:
print(env.observation_space)
print(env.action_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))
Discrete(2)


In [30]:
for i_episode in range(3):
  state = env.reset()
  while True:
    print(state)
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if done:
      print("End Game!  Reward:", reward)
      print('You Won! \n') if reward>0 else print('You Loose! \n')
      break

(15, 2, False)
(21, 2, False)
End Game!  Reward: 1.0
You Won! 

(20, 10, False)
End Game!  Reward: -1
You Loose! 

(12, 6, False)
(19, 6, False)
End Game!  Reward: 1.0
You Won! 



In [0]:
def generate_episode_from_limit_stochastic(bj_env):
  episode = []
  state = bj_env.reset()
  while True:
    probs = [0.8, 0.2] if state[0] > 18 else [0.2, 0.8]
    action = np.random.choice(np.arange(2), p=probs)
    next_state, reward, done, info = bj_env.step(action)
    episode.append((state, action, reward))
    state = next_state
    if done:
      break
  return episode

In [36]:
for i in range(7):
  print(generate_episode_from_limit_stochastic(env))

[((13, 6, True), 1, 0), ((13, 6, False), 1, 0), ((14, 6, False), 1, -1)]
[((17, 8, False), 1, 0), ((18, 8, False), 1, -1)]
[((16, 5, False), 1, -1)]
[((18, 6, False), 1, -1)]
[((16, 3, False), 0, -1.0)]
[((15, 10, False), 1, 0), ((19, 10, False), 0, 1.0)]
[((12, 5, False), 0, 1.0)]


In [0]:
def mc_prediction_q(env, num_episodes, generate_episodes, gamma=1):
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    for i in range(1, num_episodes+1):
      if i%1000 == 0:
        print('\r Episode: {}/{}'.format(i, num_episodes), end="")
        sys.stdout.flush()
    episode = generate_episodes(env)
    states, actions, rewards = zip(*episode)
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])
    for i, state in enumerate(states):
      returns_sum[states][actions[i]] = sum(rewards[i:]*discounts[:-(i+1)])
      N[states][actions[i]] += 1 
      Q[states][actions[i]] = returns_sum[states][actions[i]] / N[states][actions[i]]
    return Q

In [53]:
Q = mc_prediction_q(env, 500000, generate_episode_from_limit_stochastic)


 Episode: 500000/500000